# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4, timeoutcontroller
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1615276276.5
Main triggered 1615276277.000001
Print triggered 1615276277.000001
Main triggered 1615276277.5
Main triggered 1615276278.003628
Print triggered 1615276278.003628


# Controller Base Class (eFMU)

In [4]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 3, 'b': 22}
Output {'c': 66}
Output {'c': 66}


# Controller Stack Class (single-thread/multi-thread)

In [3]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [4]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-08 23:55:23.200111104,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:25.475936000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:27.744163072,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:30.022691072,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:32.304621056,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:34.577415936,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:36.844119040,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:39.113681920,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 23:55:41.383881984,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-08 23:55:23.200111104,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:25.475936000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:27.744163072,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:30.022691072,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:32.304621056,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:34.577415936,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:36.844119040,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:39.113681920,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 23:55:41.383881984,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-08 23:55:23.200111104,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:25.475936000,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:27.744163072,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:30.022691072,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:32.304621056,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:34.577415936,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:36.844119040,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:39.113681920,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 23:55:41.383881984,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-08 23:55:23.200111104,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:25.475936000,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:27.744163072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:30.022691072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:32.304621056,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:34.577415936,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:36.844119040,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:39.113681920,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 23:55:41.383881984,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-08 23:55:23.200111104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:25.475936000,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:27.744163072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:30.022691072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:32.304621056,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:34.577415936,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:36.844119040,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:39.113681920,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 23:55:41.383881984,30.0,4.0,120.0,testcontroller1 did a computation!


22.60380220413208


## Multi Thread

In [8]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, workers=200, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
ctrl_stack.run_query_control_for(120)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-09 00:06:04.903219968,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-09 00:06:05.928879104,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-09 00:06:06.949743104,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-09 00:06:07.956352000,10.0,4.0,40.0,testcontroller1 did a computation!
...,...,...,...,...
2021-03-09 00:07:59.846861056,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-09 00:08:00.901192960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-09 00:08:01.947787008,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-09 00:08:02.991299072,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-09 00:06:04.903219968,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-09 00:06:05.928879104,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-09 00:06:06.949743104,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-09 00:06:07.956352000,40.0,10.0,400.0,testcontroller2 did a computation!
...,...,...,...,...
2021-03-09 00:07:59.846861056,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-09 00:08:00.901192960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-09 00:08:01.947787008,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-09 00:08:02.991299072,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-09 00:06:04.903219968,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-09 00:06:05.928879104,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-09 00:06:06.949743104,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-09 00:06:08.980570112,400.0,40.0,16000.0,testcontroller3 did a computation!
...,...,...,...,...
2021-03-09 00:07:59.846861056,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-09 00:08:00.901192960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-09 00:08:01.947787008,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-09 00:08:02.991299072,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-09 00:06:04.903219968,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-09 00:06:05.928879104,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-09 00:06:07.956352000,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-09 00:06:10.006926080,20.0,4.0,80.0,testcontroller3 did a computation!
...,...,...,...,...
2021-03-09 00:07:58.794883072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-09 00:07:59.846861056,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-09 00:08:00.901192960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-09 00:08:01.947787008,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 23:55:14.646042880,NaN,NaN,NaN,Initialize
2021-03-09 00:06:04.903219968,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-09 00:06:05.928879104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-09 00:06:06.949743104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-09 00:06:07.956352000,30.0,4.0,120.0,testcontroller1 did a computation!
...,...,...,...,...
2021-03-09 00:07:59.846861056,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-09 00:08:00.901192960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-09 00:08:01.947787008,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-09 00:08:02.991299072,30.0,4.0,120.0,testcontroller1 did a computation!


122.3423261642456


In [5]:
#Adding timeoutcontroller
controllers['timeout'] = {'fun': timeoutcontroller, 'sampletime': 5}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)
print("done initializing")

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

done initializing
Controller timeout timeout


/Users/deoxi/FMLC/FMLC/stackedclasses.py:287: Warning: Controller timeout timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout timeout
Controller timeout timeout
Controller timeout timeout


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:04.718792960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:05.761614080,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:06.811237888,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:07.861688064,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:08.868992000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:09.871037952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:10.924419072,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:11.945048064,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:04.718792960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:05.761614080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:06.811237888,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:07.861688064,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:08.868992000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:09.871037952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:10.924419072,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:11.945048064,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:04.718792960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:05.761614080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:06.811237888,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:07.861688064,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:09.871037952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:10.924419072,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:12.966381056,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:14.006619904,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:04.718792960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:05.761614080,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:06.811237888,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:08.868992000,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:10.924419072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:12.966381056,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:14.006619904,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:15.051962880,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:04.718792960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:05.761614080,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:06.811237888,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:07.861688064,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:08.868992000,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:09.871037952,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:10.924419072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:11.945048064,30.0,4.0,120.0,testcontroller1 did a computation!


,a,Logging
2021-03-08 22:29:52.717533952,NaN,Initialize
2021-03-08 22:30:03.676968960,seen due to late computation,Compute ok
2021-03-08 22:30:08.711700992,seen due to late computation,Compute ok
2021-03-08 22:30:13.747479040,seen due to late computation,Compute ok
2021-03-08 22:30:18.775074048,seen due to late computation,Compute ok


22.336485862731934
